In [27]:
# 1. 라이브러리 임포트
import warnings
warnings.filterwarnings('ignore')

from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    StratifiedKFold,
    train_test_split,
)
from sklearn.metrics import classification_report, precision_recall_fscore_support, accuracy_score

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

try:
    from xgboost import XGBClassifier
except ImportError as exc:
    raise ImportError('xgboost 패키지가 필요합니다. pip install xgboost 로 설치해 주세요.') from exc

try:
    from lightgbm import LGBMClassifier
except ImportError as exc:
    raise ImportError('lightgbm 패키지가 필요합니다. pip install lightgbm 로 설치해 주세요.') from exc

print('라이브러리 임포트 완료')

라이브러리 임포트 완료


In [28]:
# 2. test.csv 로드 및 특징/타깃 분리
DATA_PATH = Path('test.csv')
if not DATA_PATH.exists():
    raise FileNotFoundError('test.csv 파일이 존재하지 않습니다. 먼저 test.ipynb에서 생성해 주세요.')

df = pd.read_csv(DATA_PATH)
print('불러온 데이터 형태:', df.shape)

if 'Target' not in df.columns:
    raise KeyError('test.csv에 Target 컬럼이 없습니다. 전처리 단계에서 Target을 포함했는지 확인하세요.')

TARGET_MAPPING = {'Dropout': 0, 'Graduate': 1, 'Enrolled': 1}
unique_targets = set(df['Target'].unique())
if not unique_targets.issubset({0, 1}):
    df['Target'] = df['Target'].map(TARGET_MAPPING)
    if df['Target'].isna().any():
        missing_labels = df.loc[df['Target'].isna(), 'Target']
        raise ValueError(f'정의되지 않은 타깃 라벨이 있습니다: {missing_labels.unique()}')

X = df.drop(columns=['Target'])
y = df['Target']

print('특징 행렬/타깃 분리 완료:')
print(' - X shape:', X.shape)
print(' - y shape:', y.shape)
print(' - y 분포:')
print(y.value_counts(normalize=True).rename('ratio'))

불러온 데이터 형태: (4424, 35)
특징 행렬/타깃 분리 완료:
 - X shape: (4424, 34)
 - y shape: (4424,)
 - y 분포:
Target
1    0.678797
0    0.321203
Name: ratio, dtype: float64


In [29]:
# 3. 학습/검증 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42,
)

print('데이터 분할 완료:')
print(' - X_train:', X_train.shape, '| X_test:', X_test.shape)
print(' - y_train 분포:')
print(y_train.value_counts(normalize=True).rename('ratio'))

데이터 분할 완료:
 - X_train: (3539, 34) | X_test: (885, 34)
 - y_train 분포:
Target
1    0.678723
0    0.321277
Name: ratio, dtype: float64


In [30]:
# 4. 모델 및 하이퍼파라미터 탐색 공간 정의
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

search_spaces = {
    'LogisticRegression': {
        'estimator': LogisticRegression(class_weight='balanced', solver='liblinear', max_iter=5000),
        'search': GridSearchCV,
        'param_grid': {
            'C': [0.01, 0.1, 1, 10, 50],
            'penalty': ['l1', 'l2'],
        },
    },
    'DecisionTree': {
        'estimator': DecisionTreeClassifier(random_state=42),
        'search': GridSearchCV,
        'param_grid': {
            'max_depth': [None, 6, 12, 20],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'class_weight': [None, 'balanced'],
        },
    },
    'RandomForest': {
        'estimator': RandomForestClassifier(random_state=42, n_jobs=-1),
        'search': RandomizedSearchCV,
        'param_grid': {
            'n_estimators': [200, 400, 600, 800],
            'max_depth': [None, 12, 20, 30],
            'min_samples_leaf': [1, 2, 4],
            'min_samples_split': [2, 5, 10],
            'max_features': ['sqrt', 'log2', 0.7],
            'class_weight': [None, 'balanced_subsample'],
        },
        'n_iter': 30,
    },
    'AdaBoost': {
        'estimator': AdaBoostClassifier(random_state=42),
        'search': GridSearchCV,
        'param_grid': {
            'n_estimators': [200, 400, 600, 800],
            'learning_rate': [0.05, 0.1, 0.25, 0.5, 1.0],
        },
    },
    'XGBoost': {
        'estimator': XGBClassifier(
            objective='binary:logistic',
            eval_metric='logloss',
            tree_method='auto',
            use_label_encoder=False,
            random_state=42,
            n_jobs=-1,
        ),
        'search': RandomizedSearchCV,
        'param_grid': {
            'n_estimators': [200, 400, 600, 800],
            'max_depth': [3, 5, 7, 9],
            'learning_rate': [0.03, 0.05, 0.1, 0.2],
            'subsample': [0.7, 0.8, 0.9, 1.0],
            'colsample_bytree': [0.6, 0.8, 1.0],
            'gamma': [0, 0.5, 1.0],
            'reg_lambda': [1.0, 2.0, 3.0, 5.0],
        },
        'n_iter': 35,
    },
    'LightGBM': {
        'estimator': LGBMClassifier(objective='binary', random_state=42, n_jobs=-1),
        'search': RandomizedSearchCV,
        'param_grid': {
            'n_estimators': [400, 600, 800, 1000],
            'learning_rate': [0.03, 0.05, 0.1],
            'num_leaves': [64, 128, 256],
            'max_depth': [-1, 10, 20, 30],
            'subsample': [0.7, 0.8, 0.9, 1.0],
            'colsample_bytree': [0.6, 0.8, 1.0],
            'reg_lambda': [0.0, 1.0, 3.0],
        },
        'n_iter': 35,
    },
}

print(f"탐색 대상 모델 수: {len(search_spaces)}")

탐색 대상 모델 수: 6


In [31]:
# 5. 하이퍼파라미터 튜닝
best_models = {}
tuning_records = []

for name, config in search_spaces.items():
    estimator = config['estimator']
    search_cls = config['search']
    param_grid = config['param_grid']

    print(f"\n=== {name} 모델 튜닝 중 ===")

    if search_cls is GridSearchCV:
        searcher = search_cls(
            estimator=estimator,
            param_grid=param_grid,
            scoring='f1',
            cv=cv,
            n_jobs=-1,
            verbose=0,
        )
    else:
        searcher = search_cls(
            estimator=estimator,
            param_distributions=param_grid,
            scoring='f1',
            cv=cv,
            n_jobs=-1,
            n_iter=config.get('n_iter', 25),
            random_state=42,
            verbose=0,
        )

    searcher.fit(X_train, y_train)
    best_models[name] = searcher.best_estimator_

    tuning_records.append(
        {
            'model': name,
            'best_cv_f1': searcher.best_score_,
            'best_params': searcher.best_params_,
        }
    )

    print(f"최적 교차검증 F1: {searcher.best_score_:.4f}")
    print('최적 하이퍼파라미터:', searcher.best_params_)


if tuning_records:
    tuning_results_df = (
        pd.DataFrame(tuning_records)
        .sort_values(by='best_cv_f1', ascending=False)
        .reset_index(drop=True)
    )
    print('\n=== 하이퍼파라미터 탐색 결과 요약 ===')
    display(tuning_results_df)
else:
    raise RuntimeError('하이퍼파라미터 탐색 결과가 비어 있습니다.')


=== LogisticRegression 모델 튜닝 중 ===
최적 교차검증 F1: 0.8983
최적 하이퍼파라미터: {'C': 1, 'penalty': 'l2'}

=== DecisionTree 모델 튜닝 중 ===
최적 교차검증 F1: 0.9056
최적 하이퍼파라미터: {'class_weight': None, 'max_depth': 6, 'min_samples_leaf': 1, 'min_samples_split': 2}

=== RandomForest 모델 튜닝 중 ===
최적 교차검증 F1: 0.9118
최적 하이퍼파라미터: {'n_estimators': 400, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 30, 'class_weight': None}

=== AdaBoost 모델 튜닝 중 ===
최적 교차검증 F1: 0.9117
최적 하이퍼파라미터: {'learning_rate': 0.25, 'n_estimators': 600}

=== XGBoost 모델 튜닝 중 ===
최적 교차검증 F1: 0.9166
최적 하이퍼파라미터: {'subsample': 1.0, 'reg_lambda': 1.0, 'n_estimators': 800, 'max_depth': 9, 'learning_rate': 0.03, 'gamma': 0.5, 'colsample_bytree': 0.6}

=== LightGBM 모델 튜닝 중 ===
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2402, number of negative: 1137
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000623 se

,model,best_cv_f1,best_params
0,XGBoost,0.916587,"{'subsample': 1.0, 'reg_lambda': 1.0, 'n_estim..."
1,LightGBM,0.913881,"{'subsample': 0.9, 'reg_lambda': 3.0, 'num_lea..."
2,RandomForest,0.911795,"{'n_estimators': 400, 'min_samples_split': 2, ..."
3,AdaBoost,0.911672,"{'learning_rate': 0.25, 'n_estimators': 600}"
4,DecisionTree,0.905571,"{'class_weight': None, 'max_depth': 6, 'min_sa..."
5,LogisticRegression,0.898343,"{'C': 1, 'penalty': 'l2'}"


In [32]:
# 6. 튜닝된 모델 평가
def evaluate_models(model_dict):
    records = []
    reports = {}

    for name, estimator in model_dict.items():
        print(f"\n=== {name} ===")
        estimator.fit(X_train, y_train)
        y_pred = estimator.predict(X_test)

        report = classification_report(
            y_test,
            y_pred,
            target_names=['Graduate', 'Dropout'],
            zero_division=0,
        )
        print(report)
        reports[name] = report

        precision, recall, f1, _ = precision_recall_fscore_support(
            y_test,
            y_pred,
            average='binary',
            zero_division=0,
        )

        records.append(
            {
                'model': name,
                'accuracy': accuracy_score(y_test, y_pred),
                'precision': precision,
                'recall': recall,
                'f1': f1,
            }
        )

    results_df = pd.DataFrame(records).sort_values(by='f1', ascending=False).reset_index(drop=True)
    best_model_name = results_df.loc[0, 'model']

    print('\n=== 모델 성능 요약 (F1 기준 정렬) ===')
    display(results_df)
    print(f"\n최적 모델: {best_model_name}")
    print('=== 최적 모델 classification_report ===')
    print(reports[best_model_name])

    return results_df, best_model_name

results_df, best_model_name = evaluate_models(best_models)


=== LogisticRegression ===
              precision    recall  f1-score   support

    Graduate       0.79      0.81      0.80       284
     Dropout       0.91      0.90      0.90       601

    accuracy                           0.87       885
   macro avg       0.85      0.85      0.85       885
weighted avg       0.87      0.87      0.87       885


=== DecisionTree ===
              precision    recall  f1-score   support

    Graduate       0.83      0.67      0.74       284
     Dropout       0.86      0.94      0.89       601

    accuracy                           0.85       885
   macro avg       0.84      0.80      0.82       885
weighted avg       0.85      0.85      0.85       885


=== RandomForest ===
              precision    recall  f1-score   support

    Graduate       0.86      0.72      0.78       284
     Dropout       0.88      0.95      0.91       601

    accuracy                           0.87       885
   macro avg       0.87      0.83      0.85       885
we

,model,accuracy,precision,recall,f1
0,LightGBM,0.874576,0.878086,0.946755,0.911129
1,RandomForest,0.873446,0.876733,0.946755,0.910400
2,XGBoost,0.870056,0.870427,0.950083,0.908512
3,AdaBoost,0.870056,0.873846,0.945092,0.908074
4,LogisticRegression,0.870056,0.909091,0.898502,0.903766
5,DecisionTree,0.850847,0.858015,0.935108,0.894904



최적 모델: LightGBM
=== 최적 모델 classification_report ===
              precision    recall  f1-score   support

    Graduate       0.86      0.72      0.79       284
     Dropout       0.88      0.95      0.91       601

    accuracy                           0.87       885
   macro avg       0.87      0.83      0.85       885
weighted avg       0.87      0.87      0.87       885

